<a href="https://colab.research.google.com/github/taehyeonk/pre-onboarding_project/blob/jina/220316_sub3_90_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 2.7 MB/s 
     |████████████████████████████████| 3.8 MB 17.4 MB/s 
     |████████████████████████████████| 1.2 MB 43.2 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 6.5 MB 36.4 MB/s 
     |████████████████████████████████| 596 kB 40.9 MB/s 
     |████████████████████████████████| 895 kB 36.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=88b82460c4d1954bd4d67e11228f6be4f9305da308815d83704cade5fb620399
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import argparse
import logging
import math
import os
import random
from datetime import datetime
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score

from sentence_transformers import SentenceTransformer, LoggingHandler, models, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator
from torch.utils.data import DataLoader

import json
from sentence_transformers.readers import InputExample

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/klue-sts-v1.1

/content/drive/MyDrive/Colab Notebooks/klue-sts-v1.1


In [ ]:
def load_kor_sts_samples(filename):
    samples = []
    with open(filename,'r') as f:
        json_data = json.load(f)
    for row in json_data:
        score = float(row['labels']['binary-label'])
        samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score, guid=row['guid']))
    return samples

In [ ]:
logging.info("Read KorSTS train/dev dataset")

train_file = './klue-sts-v1.1_train.json'
train_samples = load_kor_sts_samples(train_file)

test_file = './klue-sts-v1.1_dev.json'
test_samples =load_kor_sts_samples(test_file)

n_train = int(0.8 * len(train_samples))
valid_samples = train_samples[n_train:]
train_samples = train_samples[:n_train]

In [ ]:
# Parse command line arguments
parser = argparse.ArgumentParser()
parser.add_argument("--model_name_or_path", type=str, default = 'klue/roberta-large')
parser.add_argument("--max_seq_length", type=int, default=256)
parser.add_argument("--batch_size", type=int, default=16)
parser.add_argument("--num_epochs", type=int, default=5)
parser.add_argument("--output_dir", type=str, default="output")
parser.add_argument("--output_prefix", type=str, default="kor_sts_")
parser.add_argument("--seed", type=int, default=777)
args = parser.parse_args("")

# Fix random seed
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

# Configure logger
logging.basicConfig(
    format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S', level=logging.INFO, handlers=[LoggingHandler()]
)

# Read the dataset
model_save_path = os.path.join(
    args.output_dir,
    args.output_prefix + args.model_name_or_path.replace("/", "-") + '-' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)

In [ ]:
# Define SentenceTransformer model
word_embedding_model = models.Transformer(args.model_name_or_path, max_seq_length=args.max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device = 'cuda')
train_loss = losses.ContrastiveLoss(model=model)
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=args.batch_size)
valid_evaluator = BinaryClassificationEvaluator.from_input_examples(valid_samples, batch_size=args.batch_size,name='sts-valid',show_progress_bar=True)
warmup_steps = math.ceil(len(train_dataloader) * args.num_epochs * 0.1) 

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

...

Batches:   0%|          | 0/292 [00:00<?, ?it/s]

Batches:   0%|          | 0/292 [00:00<?, ?it/s]

Batches:   0%|          | 0/292 [00:00<?, ?it/s]

Batches:   0%|          | 0/292 [00:00<?, ?it/s]

Batches:   0%|          | 0/292 [00:00<?, ?it/s]

In [ ]:
# Define SentenceTransformer model
word_embedding_model2 = models.Transformer(args.model_name_or_path, max_seq_length=args.max_seq_length)
pooling_model2 = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)
model2 = SentenceTransformer(modules=[word_embedding_model, pooling_model], device = 'cuda')
train_loss = losses.ContrastiveLoss(model=model)
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=args.batch_size)
valid_evaluator = BinaryClassificationEvaluator.from_input_examples(valid_samples, batch_size=args.batch_size,name='sts-valid',show_progress_bar=True)
warmup_steps = math.ceil(len(train_dataloader) * args.num_epochs * 0.1) 

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [ ]:
# S-BERT adaptation 
model2.fit(train_objectives=[(train_dataloader, train_loss)],evaluation_steps=1000,
          evaluator=valid_evaluator,epochs=5,show_progress_bar=True,warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/584 [00:00<?, ?it/s]

Batches:   0%|          | 0/292 [00:00<?, ?it/s]

Iteration:   0%|          | 0/584 [00:00<?, ?it/s]

Batches:   0%|          | 0/292 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/292 [00:00<?, ?it/s]

Iteration:   0%|          | 0/584 [00:00<?, ?it/s]

Batches:   0%|          | 0/292 [00:00<?, ?it/s]

Iteration:   0%|          | 0/584 [00:00<?, ?it/s]

Batches:   0%|          | 0/292 [00:00<?, ?it/s]

In [ ]:
model2 = SentenceTransformer(model_save_path)
test_evaluator_1 = BinaryClassificationEvaluator.from_input_examples(test_samples, name='sts-test',show_progress_bar=True)
test_evaluator_1(model, output_path=model_save_path)

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

0.901709657133521

In [ ]:
#API 구현...생각했을때!
docs = [
    "1992년 7월 8일 손흥민은 강원도 춘천시 후평동에서 아버지 손웅정과 어머니 길은자의 차남으로 태어나 그곳에서 자랐다.",
    "형은 손흥윤이다.",
    "춘천 부안초등학교를 졸업했고, 춘천 후평중학교에 입학한 후 2학년때 원주 육민관중학교 축구부에 들어가기 위해 전학하여 졸업하였으며, 2008년 당시 FC 서울의 U-18팀이었던 동북고등학교 축구부에서 선수 활동 중 대한축구협회 우수선수 해외유학 프로젝트에 선발되어 2008년 8월 독일 분데스리가의 함부르크 유소년팀에 입단하였다.",
    "함부르크 유스팀 주전 공격수로 2008년 6월 네덜란드에서 열린 4개국 경기에서 4게임에 출전, 3골을 터뜨렸다.",
    "1년간의 유학 후 2009년 8월 한국으로 돌아온 후 10월에 개막한 FIFA U-17 월드컵에 출전하여 3골을 터트리며 한국을 8강으로 이끌었다.",
    "그해 11월 함부르크의 정식 유소년팀 선수 계약을 체결하였으며 독일 U-19 리그 4경기 2골을 넣고 2군 리그에 출전을 시작했다.",
    "독일 U-19 리그에서 손흥민은 11경기 6골, 2부 리그에서는 6경기 1골을 넣으며 재능을 인정받아 2010년 6월 17세의 나이로 함부르크의 1군 팀 훈련에 참가, 프리시즌 활약으로 함부르크와 정식 계약을 한 후 10월 18세에 함부르크 1군 소속으로 독일 분데스리가에 데뷔하였다.",
]
document_embeddings = model2.encode(docs)
query = "손흥민은 어린 나이에 유럽에 진출하였다."
query_embedding = model2.encode(query)

In [ ]:
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
top_k = min(5, len(docs))

# 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

# 코사인 유사도 순으로 `top_k` 개 문장 추출
top_results = torch.topk(cos_scores, k=top_k)

print(f"입력 문장: {query}")
print(f"\n<입력 문장과 유사한 {top_k} 개의 문장>\n")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    print(f"{i+1}: {docs[idx]} {'(유사도: {:.4f})'.format(score)}\n")

입력 문장: 손흥민은 어린 나이에 유럽에 진출하였다.

<입력 문장과 유사한 5 개의 문장>

1: 독일 U-19 리그에서 손흥민은 11경기 6골, 2부 리그에서는 6경기 1골을 넣으며 재능을 인정받아 2010년 6월 17세의 나이로 함부르크의 1군 팀 훈련에 참가, 프리시즌 활약으로 함부르크와 정식 계약을 한 후 10월 18세에 함부르크 1군 소속으로 독일 분데스리가에 데뷔하였다. (유사도: 0.7057)

2: 그해 11월 함부르크의 정식 유소년팀 선수 계약을 체결하였으며 독일 U-19 리그 4경기 2골을 넣고 2군 리그에 출전을 시작했다. (유사도: 0.6508)

3: 춘천 부안초등학교를 졸업했고, 춘천 후평중학교에 입학한 후 2학년때 원주 육민관중학교 축구부에 들어가기 위해 전학하여 졸업하였으며, 2008년 당시 FC 서울의 U-18팀이었던 동북고등학교 축구부에서 선수 활동 중 대한축구협회 우수선수 해외유학 프로젝트에 선발되어 2008년 8월 독일 분데스리가의 함부르크 유소년팀에 입단하였다. (유사도: 0.6082)

4: 함부르크 유스팀 주전 공격수로 2008년 6월 네덜란드에서 열린 4개국 경기에서 4게임에 출전, 3골을 터뜨렸다. (유사도: 0.5632)

5: 1992년 7월 8일 손흥민은 강원도 춘천시 후평동에서 아버지 손웅정과 어머니 길은자의 차남으로 태어나 그곳에서 자랐다. (유사도: 0.5168)

